<a href="https://colab.research.google.com/github/desstaw/DataAnonymPipeline/blob/main/K_anonymization%26DecisionTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [102]:
!pip install anonypy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
#Basic imports
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import anonypy

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier  



#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import warnings
warnings.simplefilter('ignore')

df = pd.read_csv("/content/drive/MyDrive/heart.csv")

In [104]:
#oldpeak to int
df['oldpeak'] = df['oldpeak'].astype(int) 
#categorical to object
df['sex'] = df['sex'].astype(object) 
df['cp'] = df['cp'].astype(object) 
df['fbs'] = df['fbs'].astype(object) 
df['restecg'] = df['restecg'].astype(object) 
df['exang'] = df['exang'].astype(object) 
df['slope'] = df['slope'].astype(object) 
df['ca'] = df['ca'].astype(object) 
df['thal'] = df['thal'].astype(object) 
df['target'] = df['target'].astype(int) 

In [105]:
#normalize numerical features
from sklearn.preprocessing import MinMaxScaler

df_norm = df.copy()
scaler = MinMaxScaler()
#scaler.fit(data)
df_norm[['trestbps', 'chol', 'thalach', 'oldpeak']] = scaler.fit_transform(df_norm[['trestbps', 'chol', 'thalach', 'oldpeak']])

In [106]:
print(df_norm.columns)

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


In [107]:
def k_anonymize(dataset, columns):
    """
    Anonymize certain columns of the database
    using suppression and generalization
    
    :param dataset: The set of columns and rows composing the dataset
    :param columns: Specific columns which entries are considered sensitive
    
    """
    
    anon_dataset = dataset.copy()
    
    # Target all sensitive columns
    for column in columns:
        column_label = column['label']
        if column['type'] == 'suppressed':
            # Replace all characters with asterix
            anon_dataset[column_label] = ['*' for x in anon_dataset[column_label]]
        
        if column['type'] == 'semi-suppressed':
            # Replace 70% of the characters with asterix
            anon_dataset[column_label] = [('*'*(round(len(x)*.7)) + x[(round(len(x)*.7)):]) for x in anon_dataset[column_label]]
        
        if column['type'] == 'generalized':
            # Summarize the data using ranges
            for i in range(len(anon_dataset[column_label])):
                # convert column type from int to string
                anon_dataset[column_label] = anon_dataset[column_label].astype(str)
                x = int(anon_dataset[column_label][i])
                if x <= 40: anon_dataset[column_label][i] =  "0-40"
                if x > 40 and x <= 60: anon_dataset[column_label][i] =  "40-60"
                if x > 60 and x <= 100: anon_dataset[column_label][i] =  "> 60"
            # specify the categories
            categories = ['0-40', '40-60', '> 60']
            # change the "fruit" column to the category data type
            anon_dataset['age'] = pd.Categorical(anon_dataset['age'], categories=categories)
            # convert the "fruit" column to float values using the cat.codes attribute
            anon_dataset['age'] = anon_dataset['age'].cat.codes.astype(float)
    return anon_dataset

# Describe the columns and their respective anonymization type
columns = [
    #{"label": "country", "type": "semi-suppressed"},
    {"label": "sex", "type": "suppressed"},
    #{"label": "condition", "type": "semi-suppressed"},
    {"label": "age", "type": "generalized"}
]

dataset = df_norm
anonymized_dataset = k_anonymize(dataset, columns)

In [108]:
# drop the "price" column
anonymized_dataset = anonymized_dataset.drop('sex', axis=1)

In [109]:
#anonymized_dataset['age'] = anonymized_dataset['age'].astype('object')
#anonymized_dataset.head(100)
print(anonymized_dataset.head(100))


    age cp  trestbps      chol fbs restecg   thalach exang   oldpeak slope ca  \
0   1.0  0  0.292453  0.196347   0       1  0.740458     0  0.166667     2  2   
1   1.0  0  0.433962  0.175799   1       0  0.641221     1  0.500000     0  0   
2   2.0  0  0.481132  0.109589   0       1  0.412214     1  0.333333     0  0   
3   2.0  0  0.509434  0.175799   0       1  0.687023     0  0.000000     2  1   
4   2.0  0  0.415094  0.383562   1       1  0.267176     0  0.166667     1  3   
..  ... ..       ...       ...  ..     ...       ...   ...       ...   ... ..   
95  1.0  0  0.415094  0.251142   0       0  0.618321     1  0.000000     1  0   
96  1.0  0  0.320755  0.404110   0       0  0.671756     0  0.000000     2  1   
97  1.0  0  0.273585  0.356164   0       1  0.183206     1  0.333333     1  2   
98  2.0  0  0.150943  0.278539   0       0  0.664122     0  0.000000     2  2   
99  2.0  2  0.433962  0.162100   0       2  0.343511     0  0.166667     1  0   

   thal  target  
0     3  

In [110]:
#age_scale = {'0-40':0, '40-60':1, '> 60':2}
#anonymized_dataset['age'] = anonymized_dataset.apply(lambda x: age_scale[x])

In [111]:
X = df_norm.iloc[:, df_norm.columns != 'target'].values 
y = df_norm['target'].values
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.25, random_state=4)

In [112]:
classifier= KNeighborsClassifier(n_neighbors=6)  
classifier.fit(X_train, y_train)
y_pred= classifier.predict(X_test) 

In [113]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9027237354085603


In [114]:
X = anonymized_dataset.iloc[:, anonymized_dataset.columns != 'target']
y = anonymized_dataset['target']
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.25, random_state=4)

In [115]:
classifier= KNeighborsClassifier(n_neighbors=6)  
classifier.fit(X_train, y_train)
y_pred= classifier.predict(X_test) 

In [116]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8404669260700389
